<a href="https://colab.research.google.com/github/Afia-Ali/cse445-sec5-group3-fall25/blob/Abir_2/turn_prediction_logic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# ------------------------------
# STEP 0: Install YOLOv8
# ------------------------------
!pip install ultralytics --quiet

# ------------------------------
# STEP 1: Upload & Unzip Dataset
# ------------------------------
from google.colab import files
import zipfile
import os

# Upload dataset
uploaded = files.upload()  # select Roads.v1i.folder.zip
zip_filename = list(uploaded.keys())[0]

with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
    zip_ref.extractall("dataset")

dataset_path = "dataset"
print("Dataset path:", dataset_path)

# Quick check
for split in ["train", "valid", "test"]:
    split_path = os.path.join(dataset_path, split)
    if os.path.exists(split_path):
        classes = os.listdir(split_path)
        n_imgs = sum([len(os.listdir(os.path.join(split_path, c))) for c in classes])
        print(f"{split} images: {n_imgs}, classes: {classes}")
    else:
        print(f"{split} folder not found!")

# ------------------------------
# STEP 2: Train YOLOv8 Classification Model
# ------------------------------
from ultralytics import YOLO

# Use YOLOv8 classification model
model = YOLO("yolov8n-cls.pt")  # small classification model

model.train(
    data=dataset_path,   # YOLOv8 will read class folders automatically
    epochs=50,
    batch=16,
    imgsz=224,
    project="turn_prediction",
    name="yolov8_turn_classifier",
    exist_ok=True
)

# ------------------------------
# STEP 3: Test Classification on a Single Image
# ------------------------------
import cv2
from PIL import Image as PILImage
from IPython.display import display

# Pick a test image
test_img_path = os.path.join(dataset_path, "test", "left", os.listdir(os.path.join(dataset_path,"test","left"))[0])
results = model.predict(test_img_path)

# Show results
results.show()

# Predicted class
pred_class = results[0].probs.argmax().item()  # class index
class_names = ["center", "left", "right"]      # order must match folder names
turn = class_names[pred_class]
print("Predicted turn:", turn)

# ------------------------------
# STEP 4: Predict Turns on a Video
# ------------------------------
uploaded_videos = files.upload()  # e.g., test_video.mp4
video_filename = list(uploaded_videos.keys())[0]

cap = cv2.VideoCapture(video_filename)
frames = []

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Convert frame to PIL image for classification
    from PIL import Image
    frame_pil = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

    results = model.predict(frame_pil)
    pred_class = results[0].probs.argmax().item()
    turn = class_names[pred_class]

    # Overlay turn on frame
    cv2.putText(frame, f"Turn: {turn}", (50,50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)
    frames.append(PILImage.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)))

cap.release()

# Display first frame inline
display(frames[0])

# Save video
import numpy as np
output_video = "predicted_turns_direct.mp4"
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video, fourcc, 20.0, (frames[0].width, frames[0].height))

for f in frames:
    out.write(cv2.cvtColor(np.array(f), cv2.COLOR_RGB2BGR))
out.release()
print(f"Saved output video: {output_video}")

# ------------------------------
# STEP 5: Save Model for Deployment
# ------------------------------
model.export(format="onnx")           # ONNX
model.export(format="torchscript")    # TorchScript
model.save("road_turn_model.pt")      # PyTorch


Saving Roads.v1i.folder.zip to Roads.v1i.folder.zip
Dataset path: dataset
train images: 99, classes: ['center', 'right', 'left']
valid images: 9, classes: ['center', 'right', 'left']
test images: 5, classes: ['center', 'right', 'left']
Ultralytics 8.3.236 🚀 Python-3.12.12 torch-2.9.0+cu126 CPU (AMD EPYC 7B12)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=dataset, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=224, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, mod

AttributeError: 'list' object has no attribute 'show'